In [1]:
#pip install pytest-shutil

In [19]:
import artm
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import shutil

In [3]:
df = pd.read_csv("tatarstan_message_2.csv", encoding='utf8', sep=',')

In [4]:
categories = []
for cat in df.categories:
    if cat not in categories:
        categories.append(cat)
print(categories)
print(len(categories))

['Благоустройство территории', 'Содержание и ремонт муниципальных дорог', 'Поликлиники и больницы', 'Бездомные животные', 'Парки и скверы', 'Мобильная связь', 'Организация дорожного движения', 'Незаконные азартные игры', 'Санитарное состояние', 'Нарушение в наружной рекламе', 'Работа спортивных объектов', 'Капитальный ремонт', 'Общественный транспорт', 'Воздух', 'Садоводческие, огороднические и дачные некоммерческие объединения', 'Детские сады', 'Доступная среда', 'Жилищное строительство', 'Почта', 'Вода', 'Содержание и ремонт федеральных и республиканских дорог', 'Cвалки', 'Работа учреждений культуры', 'Объекты культурного наследия', 'Сельское хозяйство', 'Цифровое телевидение']
26


In [5]:
workers = []
for w in df.worker:
    if w not in workers:
        workers.append(w)
print(workers)
print(len(workers))

['Исполнительный комитет', 'Администрация', 'Министерство здравоохранения', 'Муниципальное образование город', 'Управление административно технической инспекции', 'Комитет жилищно коммунального хозяйства', 'Комитет внешнего благоустройства', 'ООО Твои мобильные технологии', 'МБУ Дирекция парков скверов', 'Министерство внутренних дел', 'Управление Роспотребнадзор по РТрт', 'Управление наружной рекламы информации Исполнительного комитета муниципального образования города', 'Комитет по транспорту', 'Министерство спорта', 'Министерство транспорта дорожного хозяйства', 'ООО Управляющая компания Уютный дом', 'Исполнительный комитет Елабуга', 'Управление Роспотребнадзор', 'МУП Метроэлектротранс', 'Комитет земельных имущественных отношений', 'Комитет потребительского рынка', 'Государственное казенное учреждение Главное управление содержания развития дорожно транспортного комплекса при Министерстве транспорта дорожного хозяйства', 'Управление образования ИКМО', 'Комитет физической культуры спор

In [61]:
data_path = '/notebooks/artm_batches_categories'
for categorie in categories:
    print(categorie)
    model_path = os.path.join(data_path, categorie + '.dump')
    csv_path = os.path.join(data_path, categorie + '.csv')
    table_path = os.path.join(data_path, categorie + '.table.csv')
    test_path = os.path.join(data_path, 'test')
    T = 10
    model_artm = artm.ARTM(num_topics = T, topic_names = ['sbj' + str(i) for i in range(T)], class_ids = {'text':1})
    model_artm.load(model_path)
    
    shutil.rmtree(test_path)
    os.mkdir(test_path)
    batch_vectorizer_test = artm.BatchVectorizer(data_path = csv_path, data_format = 'vowpal_wabbit',
                                       target_folder = test_path, batch_size = 100)
    theta_test = model_artm.transform(batch_vectorizer=batch_vectorizer_test)
    
    worker_prob = pd.DataFrame(np.zeros((len(theta_test.index), len(workers))), index=theta_test.index, columns=workers)
    for sbj in theta_test.index:
        for item in theta_test.columns:
            if theta_test.loc[sbj][item] == theta_test[item].max():
                worker_prob.loc[sbj][df.loc[item]['worker']] += 1 #theta_test.loc[sbj][item]
    for sbj in worker_prob.index:
        summ = worker_prob.loc[sbj].sum()
        for item in worker_prob.columns:
            if summ != 0:
                worker_prob.loc[sbj][item] = worker_prob.loc[sbj][item] / summ
    worker_prob = np.round(worker_prob, 3)
    print(worker_prob)
    worker_prob.to_csv(table_path, sep = ';', index_label = 'index')

Благоустройство территории
      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.273          0.302                         0.020   
sbj1                   0.263          0.302                         0.025   
sbj2                   0.274          0.281                         0.018   
sbj3                   0.260          0.297                         0.022   
sbj4                   0.271          0.288                         0.026   
sbj5                   0.279          0.294                         0.031   
sbj6                   0.285          0.303                         0.015   
sbj7                   0.252          0.252                         0.020   
sbj8                   0.288          0.289                         0.014   
sbj9                   0.255          0.304                         0.029   

      Муниципальное образование город  \
sbj0                            0.138   
sbj1                            0.142   
sb

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.316          0.296                         0.026   
sbj1                   0.271          0.313                         0.026   
sbj2                   0.290          0.312                         0.018   
sbj3                   0.312          0.285                         0.026   
sbj4                   0.281          0.318                         0.019   
sbj5                   0.291          0.306                         0.016   
sbj6                   0.287          0.295                         0.019   
sbj7                   0.242          0.403                         0.032   
sbj8                   0.298          0.278                         0.024   
sbj9                   0.325          0.316                         0.009   

      Муниципальное образование город  \
sbj0                            0.150   
sbj1                            0.138   
sbj2                         

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.196          0.391                         0.022   
sbj1                   0.250          0.750                         0.000   
sbj2                   0.100          0.200                         0.000   
sbj3                   0.500          0.000                         0.000   
sbj4                   0.444          0.222                         0.000   
sbj5                   0.238          0.378                         0.026   
sbj6                   0.545          0.000                         0.091   
sbj7                   0.000          0.500                         0.500   
sbj8                   0.214          0.357                         0.071   
sbj9                   0.429          0.357                         0.000   

      Муниципальное образование город  \
sbj0                            0.174   
sbj1                            0.000   
sbj2                         

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.248          0.360                         0.029   
sbj1                   0.222          0.444                         0.000   
sbj2                   0.750          0.250                         0.000   
sbj3                   0.333          0.333                         0.000   
sbj4                   0.500          0.000                         0.000   
sbj5                   0.200          0.200                         0.000   
sbj6                   0.286          0.286                         0.143   
sbj7                   1.000          0.000                         0.000   
sbj8                   0.125          0.250                         0.125   
sbj9                   0.400          0.000                         0.000   

      Муниципальное образование город  \
sbj0                            0.150   
sbj1                            0.111   
sbj2                         

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.319          0.319                         0.033   
sbj1                   0.278          0.278                         0.028   
sbj2                   0.186          0.372                         0.070   
sbj3                   0.308          0.308                         0.038   
sbj4                   0.188          0.500                         0.000   
sbj5                   0.245          0.347                         0.020   
sbj6                   0.235          0.412                         0.059   
sbj7                   0.000          1.000                         0.000   
sbj8                   0.269          0.394                         0.000   
sbj9                   0.224          0.408                         0.020   

      Муниципальное образование город  \
sbj0                            0.110   
sbj1                            0.167   
sbj2                         

...

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.000           1.00                         0.000   
sbj1                   0.000           1.00                         0.000   
sbj2                   0.167           0.00                         0.000   
sbj3                   0.000           0.50                         0.000   
sbj4                   0.148           0.37                         0.037   
sbj5                   0.000           0.50                         0.000   
sbj6                   0.000           0.00                         0.000   
sbj7                   0.000           0.00                         0.000   
sbj8                   0.000           0.00                         0.000   
sbj9                   0.250           0.75                         0.000   

      Муниципальное образование город  \
sbj0                            0.000   
sbj1                            0.000   
sbj2                         

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.000          1.000                         0.000   
sbj1                   0.250          0.375                         0.000   
sbj2                   0.282          0.338                         0.038   
sbj3                   0.188          0.438                         0.000   
sbj4                   0.308          0.385                         0.000   
sbj5                   0.286          0.357                         0.071   
sbj6                   0.429          0.286                         0.000   
sbj7                   0.000          0.000                         0.000   
sbj8                   0.362          0.468                         0.000   
sbj9                   0.176          0.294                         0.000   

      Муниципальное образование город  \
sbj0                            0.000   
sbj1                            0.000   
sbj2                         

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                    0.50            0.5                           0.0   
sbj1                    0.00            0.0                           0.0   
sbj2                    1.00            0.0                           0.0   
sbj3                    1.00            0.0                           0.0   
sbj4                    0.00            1.0                           0.0   
sbj5                    0.00            0.4                           0.0   
sbj6                    0.25            0.5                           0.0   
sbj7                    0.00            0.0                           0.0   
sbj8                    1.00            0.0                           0.0   
sbj9                    0.00            0.5                           0.5   

      Муниципальное образование город  \
sbj0                              0.0   
sbj1                              0.5   
sbj2                         

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                     0.0           0.25                          0.25   
sbj1                     0.0           1.00                          0.00   
sbj2                     0.5           0.00                          0.00   
sbj3                     0.0           0.00                          0.00   
sbj4                     0.0           0.00                          0.00   
sbj5                     0.5           0.00                          0.00   
sbj6                     0.0           0.00                          0.00   
sbj7                     0.0           0.00                          0.00   
sbj8                     0.0           0.00                          0.00   
sbj9                     0.2           0.60                          0.00   

      Муниципальное образование город  \
sbj0                              0.0   
sbj1                              0.0   
sbj2                         

      Исполнительный комитет  Администрация  Министерство здравоохранения  \
sbj0                   0.000            0.0                           0.0   
sbj1                   0.000            1.0                           0.0   
sbj2                   0.000            0.0                           1.0   
sbj3                   1.000            0.0                           0.0   
sbj4                   0.000            0.0                           0.0   
sbj5                   0.000            1.0                           0.0   
sbj6                   0.333            0.0                           0.0   
sbj7                   0.000            1.0                           0.0   
sbj8                   0.000            0.0                           0.0   
sbj9                   0.125            0.5                           0.0   

      Муниципальное образование город  \
sbj0                             0.00   
sbj1                             0.00   
sbj2                         

In [77]:
#вывести топ-3 исполнителей для каждой подтемы каждой темы

for categorie in categories:
    print(categorie)
    table_path = os.path.join(data_path, categorie + '.table.csv')
    worker_prob = np.round(pd.read_csv(table_path, sep = ';', index_col = 'index'), 2)
    #print(worker_prob)
    for sbj in worker_prob.index:
        top_1_arg = np.argmax(worker_prob.loc[sbj])
        top_1 = worker_prob.loc[sbj][top_1_arg]
        worker_prob.loc[sbj][top_1_arg] = 0
        top_2_arg = np.argmax(worker_prob.loc[sbj])
        top_2 = worker_prob.loc[sbj][top_2_arg]
        worker_prob.loc[sbj][top_2_arg] = 0
        top_3_arg = np.argmax(worker_prob.loc[sbj])
        top_3 = worker_prob.loc[sbj][top_3_arg]
        print(f'Top1 {top_1}-{worker_prob.columns[top_1_arg]}, Top2 {top_2}-{worker_prob.columns[top_2_arg]}, Top3 {top_3}-{worker_prob.columns[top_3_arg]}')

Благоустройство территории
Top1 0.3-Администрация, Top2 0.27-Исполнительный комитет, Top3 0.14-Муниципальное образование город
Top1 0.3-Администрация, Top2 0.26-Исполнительный комитет, Top3 0.14-Муниципальное образование город
Top1 0.28-Администрация, Top2 0.27-Исполнительный комитет, Top3 0.15-Муниципальное образование город
Top1 0.3-Администрация, Top2 0.26-Исполнительный комитет, Top3 0.13-Муниципальное образование город
Top1 0.29-Администрация, Top2 0.27-Исполнительный комитет, Top3 0.12-Муниципальное образование город
Top1 0.29-Администрация, Top2 0.28-Исполнительный комитет, Top3 0.14-Муниципальное образование город
Top1 0.3-Администрация, Top2 0.28-Исполнительный комитет, Top3 0.14-Муниципальное образование город
Top1 0.25-Исполнительный комитет, Top2 0.25-Администрация, Top3 0.17-Муниципальное образование город
Top1 0.29-Исполнительный комитет, Top2 0.29-Администрация, Top3 0.13-Муниципальное образование город
Top1 0.3-Администрация, Top2 0.26-Исполнительный комитет, Top3 0.14-

Top1 0.33-Исполнительный комитет, Top2 0.33-Муниципальное образование город, Top3 0.17-Администрация
Top1 0.75-Администрация, Top2 0.25-Управление административно технической инспекции, Top3 0.0-Исполнительный комитет
Top1 0.0-Исполнительный комитет, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 0.5-Администрация, Top2 0.25-Исполнительный комитет, Top3 0.25-Муниципальное образование город
Top1 1.0-Исполнительный комитет, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 0.33-Исполнительный комитет, Top2 0.33-Администрация, Top3 0.33-Министерство здравоохранения
Top1 0.5-Администрация, Top2 0.5-Муниципальное образование город, Top3 0.0-Исполнительный комитет
Top1 0.0-Исполнительный комитет, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 0.33-Администрация, Top2 0.33-Муниципальное образование город, Top3 0.33-Комитет жилищно коммунального хозяйства
Top1 0.4-Исполнительный комитет, Top2 0.4-Администрация, Top3 0.2-Муниципа

Top1 1.0-Администрация, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 1.0-Администрация, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 0.5-Муниципальное образование город, Top2 0.33-Комитет жилищно коммунального хозяйства, Top3 0.17-Исполнительный комитет
Top1 0.5-Администрация, Top2 0.5-Муниципальное образование город, Top3 0.0-Исполнительный комитет
Top1 0.37-Администрация, Top2 0.26-Муниципальное образование город, Top3 0.15-Исполнительный комитет
Top1 0.5-Администрация, Top2 0.5-ООО Твои мобильные технологии, Top3 0.0-Исполнительный комитет
Top1 1.0-Комитет внешнего благоустройства, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 0.0-Исполнительный комитет, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 0.0-Исполнительный комитет, Top2 0.0-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Top1 0.75-Администрация, Top2 0.25-Исполнительный комитет, Top3 0.0-Исполнительный комитет
Cвалки